# Implementazione del task 1 - Data Transformation

In uno scenario reale, le attività preliminari fondamentali che precedono qualsiasi operazione di data mining consistono in:
1. Identificare le informazioni rilevanti per il task di data mining che si vorrà implementare.
2. Estrarre le informazioni identificate dalla fonte di dati (ad esempio, il dataset *fitbit*).
3. Integrare le informazioni estratte in un *Data Store* consistente.

Dallo stesso Data Store si potranno poi eseguire vari task di data mining (ad esempio, estrazione di regole e process mining).

Il task di *Data Preprocessing* si occupa quindi di selezionare, trasformare e caricare i dati in un Data Store (o Prepared Data) che possa gestire formati di dati specifici per vari task di data mining.

## Struttura del dataset fitbit
Il sito [https://datasets.simula.no/pmdata/](https://datasets.simula.no/pmdata/) presenta tutti i riferimenti sulla struttura del dataset fitbit.

Ognuno dei 16 partecipanti presenti nel dataset ha una cartella `pNUMBER` associata che contiene varie informazioni relative ad un periodo di sei mesi durante il quale ha accettato di auto-monitorarsi. Ai fini della nostra analisi verranno utilizzati i file contenuti nelle cartelle `pNUMBER/fitbit` e `pNUMBER/pmsys`.

In [1]:
# IMPORT
import pandas as pd
import json as json
import datetime as dt
import dateutil as du
import csv

In [2]:
# VARIABILI
PATH = './pmdata/'
people = range(1,17)  #[1,2,3]

In [3]:
# FUNZIONI DI SUPPORTO
# Questa funzione serve a normalizzare il dataset exercise sostituendo i NaN con il valore medio della colonna
def fill_NaN_for_excercise_0(ex_0):
    ex_0 = ex_0.copy()
    # dall'analisi fatta sui dati sappiamo già quali sono le colonne
    column = ['elavation_gain', 'steps', 'average_heart_rate']
    means = []
    for col in column:
        if col != 'elavation_gain':
            means.append(int(ex_0[col].mean()))
        else:
            means.append(ex_0[col].mean())
    
    for i, col in enumerate(column):
        ex_0[col].fillna(value=means[i], inplace=True)
    return ex_0

In [4]:
# FUNZIONI (per i file json)
def calories_to_df(root_path, partecipants):
    dfs = []
    for p_id in partecipants: 
        p_folder = root_path + "p{:02d}".format(p_id) + '/fitbit/'
        calories_file = 'calories.json'
        
        # Leggiamo il file json per trasformarlo in un dataframe
        with open(p_folder + calories_file) as file:
            dict_cal = json.load(file)
        
        for d in dict_cal:
            # Convertiamo i dati nei loro tipi appropriati e assegniamo loro dei nomi significativi
            d['TS'] = du.parser.parse(d['dateTime'])
            d['calories'] = float(d['value'])
            d.pop('dateTime')
            d.pop('value')
            
        # Trasformiamo il dizionario appena costruito in un dataframe
        df_cal = pd.DataFrame.from_dict(dict_cal)
        df_cal['partecipant'] = p_id
        df_cal = df_cal.set_index(['partecipant','TS'])
        dfs.append(df_cal)
    r = pd.concat(dfs)  # unisce i diversi dataframe in uno unico
    r = r.sort_index()  # ordina il dataframe rispetto all'indice formato da partecipante e TS
    return r

def sedentary_minutes_to_df(root_path, partecipants):
    dfs = []
    for p_id in partecipants: 
        p_folder = root_path + "p{:02d}".format(p_id) + '/fitbit/'
        file = 'sedentary_minutes.json'
        with open(p_folder + file) as file:
            dictionary = json.load(file)
        for d in dictionary:
            d['TS'] = du.parser.parse(d['dateTime'])
            d['sedentary_minutes'] = int(d['value'])
            d.pop('dateTime')
            d.pop('value')
        df = pd.DataFrame.from_dict(dictionary)
        df['partecipant'] = p_id
        df = df.set_index(['partecipant','TS'])
        dfs.append(df)
    r = pd.concat(dfs)
    r = r.sort_index()
    return r

def distance_to_df(root_path, partecipants):
    dfs = []
    for p_id in partecipants: 
        p_folder = root_path + "p{:02d}".format(p_id) + '/fitbit/'
        file = 'distance.json'
        with open(p_folder + file) as file:
            dictionary = json.load(file)
        for d in dictionary:
            d['TS'] = du.parser.parse(d['dateTime'])
            d['distance'] = int(d['value'])
            d.pop('dateTime')
            d.pop('value')
        df = pd.DataFrame.from_dict(dictionary)
        df['partecipant'] = p_id
        df = df.set_index(['partecipant','TS'])
        dfs.append(df)
    r = pd.concat(dfs)
    r = r.sort_index()
    return r

def heart_rate_to_df(root_path, partecipants):
    dfs = []
    for p_id in partecipants: 
        p_folder = root_path + "p{:02d}".format(p_id) + '/fitbit/'
        file = 'heart_rate.json'
        with open(p_folder + file) as file:
            dictionary = json.load(file)
        for d in dictionary:
            d['TS'] = du.parser.parse(d['dateTime'])
            d['bpm'] = int(d['value']['bpm'])
            d['confidence'] = int(d['value']['confidence'])
            d.pop('dateTime')
            d.pop('value')
        df = pd.DataFrame.from_dict(dictionary)
        df['partecipant'] = p_id
        df = df.set_index(['partecipant','TS'])
        dfs.append(df)
    r = pd.concat(dfs)
    r = r.sort_index()
    return r

def steps_to_df(root_path, partecipants):
    dfs = []
    for p_id in partecipants: 
        p_folder = root_path + "p{:02d}".format(p_id) + '/fitbit/'
        file = 'steps.json'
        with open(p_folder + file) as file:
            dictionary = json.load(file)
        for d in dictionary:
            d['TS'] = du.parser.parse(d['dateTime'])
            d['steps'] = int(d['value'])
            d.pop('dateTime')
            d.pop('value')
        df = pd.DataFrame.from_dict(dictionary)
        df['partecipant'] = p_id
        df = df.set_index(['partecipant','TS'])
        dfs.append(df)
    r = pd.concat(dfs)
    r = r.sort_index()
    return r

def lightly_active_minutes_to_df(root_path, partecipants):
    dfs = []
    for p_id in partecipants: 
        p_folder = root_path + "p{:02d}".format(p_id) + '/fitbit/'
        file = 'lightly_active_minutes.json'
        
        # provo ad aprire il file, se non c'è salto quel partecipante
        try:
            _ = open(p_folder + file)
        except:
            continue
        
        with open(p_folder + file) as file:
            dictionary = json.load(file)
        for d in dictionary:
            d['TS'] = du.parser.parse(d['dateTime'])
            d['lightly_active_minutes'] = int(d['value'])
            d.pop('dateTime')
            d.pop('value')
        df = pd.DataFrame.from_dict(dictionary)
        df['partecipant'] = p_id
        df = df.set_index(['partecipant','TS'])
        dfs.append(df)
    r = pd.concat(dfs)
    r = r.sort_index()
    return r

def time_in_heart_rate_zones_to_df(root_path, partecipants):
    dfs = []
    for p_id in partecipants: 
        p_folder = root_path + "p{:02d}".format(p_id) + '/fitbit/'
        file = 'time_in_heart_rate_zones.json'
        with open(p_folder + file) as file:
            dictionary = json.load(file)
        for d in dictionary:
            d['TS'] = du.parser.parse(d['dateTime'])
            d['BELOW_DEFAULT_ZONE_1'] = float(d['value']['valuesInZones']['BELOW_DEFAULT_ZONE_1'])
            d['IN_DEFAULT_ZONE_1'] = float(d['value']['valuesInZones']['IN_DEFAULT_ZONE_1'])
            d['IN_DEFAULT_ZONE_2'] = float(d['value']['valuesInZones']['IN_DEFAULT_ZONE_2'])
            d['IN_DEFAULT_ZONE_3'] = float(d['value']['valuesInZones']['IN_DEFAULT_ZONE_3'])
            d.pop('dateTime')
            d.pop('value')
        df = pd.DataFrame.from_dict(dictionary)
        df['partecipant'] = p_id
        df = df.set_index(['partecipant','TS'])
        dfs.append(df)
    r = pd.concat(dfs)
    r = r.sort_index()
    return r

def moderately_active_minutes_to_df(root_path, partecipants):
    dfs = []
    for p_id in partecipants: 
        p_folder = root_path + "p{:02d}".format(p_id) + '/fitbit/'
        file = 'moderately_active_minutes.json'
        with open(p_folder + file) as file:
            dictionary = json.load(file)
        for d in dictionary:
            d['TS'] = du.parser.parse(d['dateTime'])
            d['moderately_active_minutes'] = int(d['value'])
            d.pop('dateTime')
            d.pop('value')
        df = pd.DataFrame.from_dict(dictionary)
        df['partecipant'] = p_id
        df = df.set_index(['partecipant','TS'])
        dfs.append(df)
    r = pd.concat(dfs)
    r = r.sort_index()
    return r

def very_active_minutes_to_df(root_path, partecipants):
    dfs = []
    for p_id in partecipants: 
        p_folder = root_path + "p{:02d}".format(p_id) + '/fitbit/'
        file = 'very_active_minutes.json'
        with open(p_folder + file) as file:
            dictionary = json.load(file)
        for d in dictionary:
            d['TS'] = du.parser.parse(d['dateTime'])
            d['very_active_minutes'] = int(d['value'])
            d.pop('dateTime')
            d.pop('value')
        df = pd.DataFrame.from_dict(dictionary)
        df['partecipant'] = p_id
        df = df.set_index(['partecipant','TS'])
        dfs.append(df)
    r = pd.concat(dfs)
    r = r.sort_index()
    return r

def resting_heart_rate_to_df(root_path, partecipants):
    dfs = []
    for p_id in partecipants: 
        p_folder = root_path + "p{:02d}".format(p_id) + '/fitbit/'
        file = 'resting_heart_rate.json'
        
        # provo ad aprire il file, se non c'è salto quel partecipante
        try:
            _ = open(p_folder + file)
        except:
            continue
        
        with open(p_folder + file) as file:
            dictionary = json.load(file)
        for d in dictionary:
            d['TS'] = du.parser.parse(d['dateTime'])
            d['resting_heart_rate'] = float(d['value']['value'])
            d['resting_heart_rate_error'] = float(d['value']['error'])
            d.pop('dateTime')
            d.pop('value')
        df = pd.DataFrame.from_dict(dictionary)
        df['partecipant'] = p_id
        df = df.set_index(['partecipant','TS'])
        dfs.append(df)
    r = pd.concat(dfs)
    r = r.sort_index()
    return r

def exercise_to_df(root_path, partecipants):
    # abbiamo due classi e ritorniamo due DataFrame
    dfs = [[], []]
    for p_id in partecipants: 
        p_folder = root_path + "p{:02d}".format(p_id) + '/fitbit/'
        file = 'exercise.json'
        with open(p_folder + file) as file:
            dictionary = json.load(file)
        
        # dividiamo il dataset in due gruppi
        final_dict_list = [[], []]
        for d in dictionary:
            final_dict = {}
            final_dict['TS'] = du.parser.parse(d['startTime'])
            final_dict['activity_name'] = d['activityName']
            final_dict['activity_level_sedentary_minutes'] = int(d['activityLevel'][0]['minutes'])
            final_dict['activity_level_lightly_minutes'] = int(d['activityLevel'][1]['minutes'])
            final_dict['activity_level_fairly_minutes'] = int(d['activityLevel'][2]['minutes'])
            final_dict['activity_level_very_minutes'] = int(d['activityLevel'][3]['minutes'])
            final_dict['calories'] = int(d['calories'])
            final_dict['active_duration'] = int(d['activeDuration'])
            
            # questo tipo di implementazione deriva dagli studi fatti sul dataset che hanno rilevato questo:
            # la maggior parte (più del 99%) delle istanze che non hanno i valori come
            # 'activity_level_sedentary_minutes' non hanno neanche 'elevationGain', 'steps' e 'averageHeartRate'.
            # Per cui noi li inseriamo solo per le instanze che hanno i valori come 'activity_level_sedentary_minutes'.
            try:
                _class = 0
                
                #"heartRateZones": [{"name": "Out of Range", "min": 30, "max": 86, "minutes": 1},
                #{"name": "Fat Burn", "min": 86, "max": 121, "minutes": 20},
                #{"name": "Cardio", "min": 121, "max": 147, "minutes": 0},
                #{"name": "Peak", "min": 147, "max": 220, "minutes": 0}]
                final_dict['hrz_out_of_range_minutes'] = int(d['heartRateZones'][0]['minutes'])
                final_dict['hrz_out_of_range_min'] = int(d['heartRateZones'][0]['min'])
                final_dict['hrz_out_of_range_max'] = int(d['heartRateZones'][0]['max'])
                
                final_dict['hrz_fat_burn_minutes'] = int(d['heartRateZones'][1]['minutes'])
                final_dict['hrz_fat_burn_min'] = int(d['heartRateZones'][1]['min'])
                final_dict['hrz_fat_burn_max'] = int(d['heartRateZones'][1]['max'])
                
                final_dict['hrz_cardio_minutes'] = int(d['heartRateZones'][2]['minutes'])
                final_dict['hrz_cardio_min'] = int(d['heartRateZones'][2]['min'])
                final_dict['hrz_cardio_max'] = int(d['heartRateZones'][2]['max'])
                
                final_dict['hrz_peak_minutes'] = int(d['heartRateZones'][3]['minutes'])
                final_dict['hrz_peak_min'] = int(d['heartRateZones'][3]['min'])
                final_dict['hrz_peak_max'] = int(d['heartRateZones'][3]['max'])

                # Nota: non tutte le attività hanno elevationGain, steps ecc. Per cui proviamo ad inserirlo e se non
                #       si riesce lo impostiamo a None.
                try:
                    final_dict['elavation_gain'] = float(d['elevationGain'])
                except:
                    final_dict['elavation_gain'] = None

                try:
                    final_dict['steps'] = int(d['steps'])
                except:
                    final_dict['steps'] = None

                try:
                    final_dict['average_heart_rate'] = int(d['averageHeartRate'])
                except:
                    final_dict['average_heart_rate'] = None

            except:
                _class = 1

            final_dict_list[_class].append(final_dict)
        
        # per le due classi
        df_class_0 = pd.DataFrame.from_dict(final_dict_list[0])
        df_class_1 = pd.DataFrame.from_dict(final_dict_list[1])
        
        # per eseguire l'aggiunta della colonna e l'impostazione dell'indice controllo prima se il dataset ha delle entry
        # altrimenti darà errore.

        if len(df_class_0) > 0:
            df_class_0['partecipant'] = p_id
            df_class_0 = df_class_0.set_index(['partecipant','TS'])
            dfs[0].append(df_class_0)
        if len(df_class_1) > 0:
            df_class_1['partecipant'] = p_id
            df_class_1 = df_class_1.set_index(['partecipant','TS'])
            dfs[1].append(df_class_1)
    
    # normalizzo exercise_0. La funzione agisce direttamente sul dataset e quindi la funzione non restituisce niente
    return fill_NaN_for_excercise_0(pd.concat(dfs[0]).sort_index()), pd.concat(dfs[1]).sort_index()
    

def sleep_to_df(root_path, partecipants):
    # abbiamo due classi e ritorniamo due DataFrame
    dfs = [[],[]]
    for p_id in partecipants: 
        p_folder = root_path + "p{:02d}".format(p_id) + '/fitbit/'
        file = 'sleep.json'
        with open(p_folder + file) as file:
            dictionary = json.load(file)
        
        # dividiamo il dataset in due gruppi:
        # - il primo dataset comprende solo coloro che hanno dormito
        # - il secondo dataset comprende coloro che si sono svegliati durante la notte
        final_dict_list = [[], []]
        for d in dictionary:
            final_dict = {}
            final_dict['TS'] = du.parser.parse(d['startTime'])
            final_dict['duration'] = d['duration']
            final_dict['minutes_to_fall_asleep'] = int(d['minutesToFallAsleep'])
            final_dict['minutes_asleep'] = int(d['minutesAsleep'])
            final_dict['minutes_awake'] = int(d['minutesAwake'])
            final_dict['minutes_after_wakeup'] = int(d['minutesAfterWakeup'])
            final_dict['efficiency'] = int(d['efficiency'])
            final_dict['main_sleep'] = bool(d['mainSleep'])

            try:
                _class = 0
                final_dict['level_deep_count'] = int(d['levels']['summary']['deep']['count'])
                final_dict['level_deep_minutes'] = int(d['levels']['summary']['deep']['minutes'])
                
                final_dict['level_wake_count'] = int(d['levels']['summary']['wake']['count'])
                final_dict['level_wake_minutes'] = int(d['levels']['summary']['wake']['minutes'])
                
                final_dict['level_light_count'] = int(d['levels']['summary']['light']['count'])
                final_dict['level_light_minutes'] = int(d['levels']['summary']['light']['minutes'])
                
                final_dict['level_rem_counts'] = int(d['levels']['summary']['rem']['count'])
                final_dict['level_rem_minutes'] = int(d['levels']['summary']['rem']['minutes'])
            except:
                _class = 1
                final_dict['level_restless_count'] = int(d['levels']['summary']['restless']['count'])
                final_dict['level_restless_minutes'] = int(d['levels']['summary']['restless']['minutes'])
                
                final_dict['level_awake_count'] = int(d['levels']['summary']['awake']['count'])
                final_dict['level_awake_minutes'] = int(d['levels']['summary']['awake']['minutes'])
                
                final_dict['level_asleep_count'] = int(d['levels']['summary']['asleep']['count'])
                final_dict['level_asleep_minutes'] = int(d['levels']['summary']['asleep']['minutes'])

            final_dict_list[_class].append(final_dict)
        
        # per le due classi
        df_class_0 = pd.DataFrame.from_dict(final_dict_list[0])
        df_class_1 = pd.DataFrame.from_dict(final_dict_list[1])
        
        # per eseguire l'aggiunta della colonna e l'impostazione dell'indice controllo prima se il dataset ha delle entry
        # altrimenti darà errore.

        if len(df_class_0) > 0:
            df_class_0['partecipant'] = p_id
            df_class_0 = df_class_0.set_index(['partecipant','TS'])
            dfs[0].append(df_class_0)
        if len(df_class_1) > 0:
            df_class_1['partecipant'] = p_id
            df_class_1 = df_class_1.set_index(['partecipant','TS'])
            dfs[1].append(df_class_1)
    
    return pd.concat(dfs[0]).sort_index(), pd.concat(dfs[1]).sort_index()

In [5]:
# FUNZIONI (per i file csv)
def srpe_to_df(root_path, partecipants):
    dfs = []
    for p_id in partecipants: 
        p_folder = root_path + "p{:02d}".format(p_id) + '/pmsys/'
        file = 'srpe.csv'
        with open(p_folder + file) as file:
            lines = csv.reader(file, delimiter=',')
        
            first_line = True
            final_dict_list = []
            for l in lines:
                if first_line:
                    first_line = False
                    continue
                else:
                    final_dict = {}
                    final_dict['TS'] = du.parser.parse(l[0])
                    final_dict['activity_names'] = list(eval(l[1]))
                    final_dict['perceived_exertion'] = int(l[2]) if l[2] != '' else None
                    final_dict['duration_min'] = int(l[3]) if l[3] != '' else None
                    
                    # le righe che hanno colonne vuote non verranno inserite nel df (vedi p16)
                    if len(final_dict['activity_names']) > 0:
                        final_dict_list.append(final_dict)

        if len(final_dict_list) > 0:
            df = pd.DataFrame.from_dict(final_dict_list)
            df['partecipant'] = p_id
            df = df.set_index(['partecipant','TS'])
            dfs.append(df)
    r = pd.concat(dfs)
    r = r.sort_index()
    return r

def wellness_to_df(root_path, partecipants):
    dfs = []
    for p_id in partecipants: 
        p_folder = root_path + "p{:02d}".format(p_id) + '/pmsys/'
        file = 'wellness.csv'
        with open(p_folder + file) as file:
            lines = csv.reader(file, delimiter=',')
        
            first_line = True
            final_dict_list = []
            for l in lines:
                if first_line:
                    first_line = False
                    continue
                else:
                    final_dict = {}
                    final_dict['TS'] = du.parser.parse(l[0])
                    final_dict['fatigue'] = int(l[1])
                    final_dict['mood'] = int(l[2])
                    final_dict['readiness'] = int(l[3])
                    final_dict['sleep_duration_h'] = int(l[4])
                    final_dict['sleep_quality'] = int(l[5])
                    final_dict['soreness'] = int(l[6])
                    final_dict['soreness_area'] = list(eval(l[7]))
                    final_dict['stress'] = int(l[8])
                    final_dict_list.append(final_dict)

        df = pd.DataFrame.from_dict(final_dict_list)
        df['partecipant'] = p_id
        df = df.set_index(['partecipant','TS'])
        dfs.append(df)
    r = pd.concat(dfs)
    r = r.sort_index()
    return r

def injury_to_df(root_path, partecipants):
    dfs = []
    for p_id in partecipants: 
        p_folder = root_path + "p{:02d}".format(p_id) + '/pmsys/'
        file = 'injury.csv'
        
        # provo ad aprire il file, se non c'è salto quel partecipante
        try:
            _ = open(p_folder + file)
        except:
            continue
        
        with open(p_folder + file) as file:
            lines = csv.reader(file, delimiter=',')
        
            first_line = True
            final_dict_list = []
            for l in lines:
                if first_line:
                    first_line = False
                    continue
                else:
                    inj_dict = dict(eval(l[1]))
                    
                    # creo tante righe quanti sono gli infortuni (avranno lo stesso TS)
                    for inj in inj_dict:                        
                        final_dict = {}
                        final_dict['TS'] = du.parser.parse(l[0])
                        final_dict['injury'] = inj
                        final_dict['severity'] = inj_dict[inj]
                        final_dict_list.append(final_dict)

        if len(final_dict_list) > 0:
            df = pd.DataFrame.from_dict(final_dict_list)
            df['partecipant'] = p_id
            df = df.set_index(['partecipant','TS'])
            dfs.append(df)
    r = pd.concat(dfs)
    r = r.sort_index()
    return r

## Data Transformation

Richiamiamo le funzioni create per ciascun file del tipo:
- calories.json
- sedentary_minutes.json
- distance.json
- sleep.json
- exercise.json
- heart_rate.json
- steps.json
- lightly_active_minutes.json
- time_in_heart_rate_zones.json
- moderately_active_minutes.json
- very_active_minutes.json
- resting_heart_rate.json
- injury.csv
- srpe.csv
- wellness.csv

Tali funzioni sono state adattate in seguito all'ispezione dei campi che ci ha permesso di identificare le informazioni più rilevanti. Ad esempio, nei file del tipo `resting_heart_rate` era presente un campo con una data ridondante.

In [6]:
sedentary_minutes = sedentary_minutes_to_df(PATH, people)

sleep_0, sleep_1 = sleep_to_df(PATH, people)
exercise_0, exercise_1 = exercise_to_df(PATH, people)

lightly_active_minutes = lightly_active_minutes_to_df(PATH, people)
time_in_heart_rate_zones = time_in_heart_rate_zones_to_df(PATH, people)
moderately_active_minutes = moderately_active_minutes_to_df(PATH, people)
very_active_minutes = very_active_minutes_to_df(PATH, people)
resting_heart_rate = resting_heart_rate_to_df(PATH, people)

In [7]:
srpe = srpe_to_df(PATH, people)
wellness = wellness_to_df(PATH, people)
injury = injury_to_df(PATH, people)

### Creiamo e poi salviamo i dataframe più grossi

Verranno posizionati nella directory `dataframes`.

In [ ]:
steps = steps_to_df(PATH, people)
distance = distance_to_df(PATH, people)
calories = calories_to_df(PATH, people)
heart_rate = heart_rate_to_df(PATH, people)

steps.to_pickle("dataframes/steps.pkl")
distance.to_pickle("dataframes/distance.pkl")
calories.to_pickle("dataframes/calories.pkl")
heart_rate.to_pickle("dataframes/heart_rate.pkl")

### Studio degli eventi Exercise e Sleep

Guardando le righe riguardanti l'aver fatto o meno esercizio si osserva che quelle che hanno fatto esercizio hanno più del 90% di probabilità di avere tre attributi considerati problematici (in quanto non tutte le attività presentano un valore per `elavation_gain`, `steps` e `average_heart_rate`). Per le righe che non hanno fatto esercizio, inoltre, si osserva che la maggior parte delle volte esse non presentano il valore per questi tre attributi. Quindi, negli altri casi, si è deciso di procedere a togliere direttamente i valori perché posseduti dalla minoranza delle righe. Alla fine avremo perciò due dataset: il primo ha le righe che hanno fatto esercizio, mentre il secondo ha quelle che non hanno fatto esercizio.

Per quanto riguarda il file `sleep` invece, siccome esso contiene una suddivisione del sonno in periodi (come sonno leggero, sonno profondo, fase REM, veglia), si è deciso di scomporlo in due dataframe:
- Il primo comprende le fasi di coloro che hanno dormito (deep, wake, light, rem).
- Il secondo comprende le fasi di coloro che si sono svegliati durante la notte (restless, awake, asleep).

In [ ]:
# import numpy as np
# people = [i for i in range(1,17)]

# DA ESEGUIRE CON COPY_sleep_to_df
# sleep = sleep_to_df(PATH, people)
# index = []
# for i in range(len(sleep)):
#     if not np.isnan(sleep.iloc[i]['level_rem_minutes']):
#         # controllo se ci sono righe in cui ho entrambi le classi
#         if not np.isnan(sleep.iloc[i]['level_restless_count']):
#             index.append(i)
# print(len(index) / len(sleep) * 100)

# DA ESEGUIRE CON COPY_exercise_to_df
# exercise = exercise_to_df(PATH, people)
# index = []
# for i in range(len(exercise)):
#     if np.isnan(exercise.iloc[i]['activity_level_sedentary_minutes']):
#         # controllo se ci sono righe in cui ho entrambi le classi
#         if not np.isnan(exercise.iloc[i]['average_heart_rate']):
#             index.append(i)
# print(len(index) / len(exercise) * 100)


#                                     Sommario dati:

# ( activity_level_sedentary_minutes != NaN ) AND (elavation_gain == NaN) -----> 6.1%
# ( activity_level_sedentary_minutes != NaN ) AND (elavation_gain != NaN) -----> 93.9%

# #######################################################################################
# ( activity_level_sedentary_minutes == NaN ) AND (elavation_gain == NaN) -----> 99.75%
# ( activity_level_sedentary_minutes == NaN ) AND (elavation_gain != NaN) -----> 0.25%

# #######################################################################################
# ( activity_level_sedentary_minutes != NaN ) AND (steps == NaN) -----> 6.7%
# ( activity_level_sedentary_minutes != NaN ) AND (steps != NaN) -----> 92.7%

# #######################################################################################
# ( activity_level_sedentary_minutes == NaN ) AND (steps == NaN) -----> 99.64%
# ( activity_level_sedentary_minutes == NaN ) AND (steps != NaN) -----> 0.36%

# #######################################################################################
# ( activity_level_sedentary_minutes != NaN ) AND (average_heart_rate == NaN) -----> 0.62%
# ( activity_level_sedentary_minutes != NaN ) AND (average_heart_rate != NaN) -----> 99.38%

# #######################################################################################
# ( activity_level_sedentary_minutes == NaN ) AND (average_heart_rate == NaN) -----> 99.919%
# ( activity_level_sedentary_minutes == NaN ) AND (average_heart_rate != NaN) -----> 0.081%